In [0]:
from databricks.labs.dqx.profiler.profiler import DQProfiler
from databricks.labs.dqx.profiler.generator import DQGenerator
from databricks.labs.dqx.profiler.dlt_generator import DQDltGenerator
from databricks.labs.dqx.engine import DQEngine
from databricks.sdk import WorkspaceClient
import yaml
from databricks.labs.dqx.contexts.workspace import WorkspaceContext



In [0]:
# Initialize the WorkspaceClient to interact with the Databricks workspace
ws = WorkspaceClient()

# Initialize a DQProfiler instance with the workspace client
profiler = DQProfiler(ws)

generator = DQGenerator(ws)

dq_engine = DQEngine(ws)


ws = WorkspaceClient()
profiler = DQProfiler(ws)
generator = DQGenerator(ws)
dlt_generator = DQDltGenerator(ws)
dq_engine = DQEngine(ws)


In [0]:
with open( "custom_dqx_rules.yaml", "r") as file:
    check_dict = yaml.safe_load(file)

dq_engine = DQEngine(spark)
validation_result = dq_engine.validate_checks(check_dict)

assert not validation_result.has_errors, f"Validation failed: {validation_result.errors}"

silver_df, quarantine_df = dq_engine.apply_checks_by_metadata_and_split(input_df, check_dict)

In [0]:
display(quarantine_df)

In [0]:
# Create a DQEngine instance with the WorkspaceClient
dq_engine = DQEngine(WorkspaceClient())

# Apply quality checks and split the DataFrame into silver and quarantine DataFrames
silver_df, quarantine_df = dq_engine.apply_checks_by_metadata_and_split(input_df, check_dict)

In [0]:
display(quarantine_df.count())
display(silver_df.count())
display(input_df.count())

In [0]:
ctx = WorkspaceContext(WorkspaceClient())
dashboards_folder_link = f"{ctx.installation.workspace_link('https://adb-8333330282859393.13.azuredatabricks.net/')}dashboards/"
print(f"Open a dashboard from the following folder and refresh it:")
print(dashboards_folder_link)

In [0]:
# Initialize the WorkspaceClient to interact with the Databricks workspace
ws = WorkspaceClient()

# Initialize a DQProfiler instance with the workspace client
profiler = DQProfiler(ws)

# Read the input data from a Delta table
input_df = spark.read.csv("dbfs:/databricks-datasets/flights/departuredelays.csv", header=True, inferSchema=True)


# Display a sample of the input data
input_df.display()

In [0]:
input_df = spark.read.csv("dbfs:/databricks-datasets/flights/departuredelays.csv", header=True, inferSchema=True)
summary_stats, profiles = profiler.profile(input_df, opts={"sample_fraction": 1.0})
print(yaml.safe_dump(summary_stats))

In [0]:
for profile in profiles:
    print('*',profile)

In [0]:
generator = DQGenerator(ws)
checks = generator.generate_dq_rules(profiles)  # with default level "error"
print(yaml.safe_dump(checks))

In [0]:
dlt_generator = DQDltGenerator(ws)

dlt_expectations = dlt_generator.generate_dlt_rules(profiles, language="SQL")
print(dlt_expectations)

dlt_expectations = dlt_generator.generate_dlt_rules(profiles, language="Python")
print(dlt_expectations)

dlt_expectations = dlt_generator.generate_dlt_rules(profiles, language="Python_Dict")
print(dlt_expectations)

In [0]:
user_name = spark.sql("select current_user() as user").collect()[0]["user"]
checks_file = f"/Workspace/Users/{user_name}/dqx_demo_checks.yml"
dq_engine = DQEngine(ws)
dq_engine.save_checks_in_workspace_file(checks=checks, workspace_path=checks_file)

In [0]:
dq_engine.save_checks_in_table(checks=checks, table_name="main.default.dqx_checks_table", mode="overwrite")

In [0]:
# Databricks Notebook Source
# MAGIC %md
# MAGIC # Databricks DQX Showcase: Flight Departure Delays Data Quality
# MAGIC
# MAGIC This notebook demonstrates how to use Databricks DQX (Data Quality eXtensions) to define, run, and visualize data quality checks on the `departuredelays.csv` dataset.
# MAGIC
# MAGIC We'll focus on validating columns like `date`, `delay`, `distance`, `origin`, and `destination`.
# MAGIC
# MAGIC **DQX is an open-source project from Databricks Labs.** This means it's community-driven and does not come with official Databricks customer support like core products. Support is primarily via GitHub issues and community forums.

# COMMAND ----------
# MAGIC %md
# MAGIC ## 1. Install Databricks DQX
# MAGIC
# MAGIC First, ensure DQX is installed in your cluster environment.

# COMMAND ----------

#pip install databricks-dqx

# COMMAND ----------
# MAGIC %md
# MAGIC ## 2. Load the Dataset
# MAGIC
# MAGIC We'll load the default `dbfs:/databricks-datasets/flights/departuredelays.csv` dataset.

# COMMAND ----------

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

spark = SparkSession.builder.appName("DQX_Demo").getOrCreate()

# Load the dataset
df = spark.read \
  .format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .load("dbfs:/databricks-datasets/flights/departuredelays.csv")

# Cast 'date' to a proper date type for date-related checks
df = df.withColumn("date", to_date(col("date").cast("string"), "MMddHHmm"))

# Create a temporary view for DQX to work with SQL expressions
df.createOrReplaceTempView("departure_delays")

print(f"Loaded DataFrame with {df.count()} rows and {len(df.columns)} columns.")
df.printSchema()
df.display()

# COMMAND ----------
# MAGIC %md
# MAGIC ## 3. Initialize DQX API
# MAGIC
# MAGIC We need to create an instance of the `DataQualityAPI` to start defining and running checks.

# COMMAND ----------

from databricks.dqx.api.data_quality_api import DataQualityAPI
from databricks.dqx.checks.check import Check, CheckType

dq_api = DataQualityAPI()

# COMMAND ----------
# MAGIC %md
# MAGIC ## 4. Define Data Quality Checks
# MAGIC
# MAGIC Here, we'll define various data quality checks using DQX's declarative syntax. We'll cover common checks for our flight delays dataset columns.
# MAGIC
# MAGIC **Columns:** `date`, `delay`, `distance`, `origin`, `destination`

# COMMAND ----------

# Define a list to hold our data quality checks
checks = []

# --- General Checks ---
# Check for duplicate rows in the entire dataset
checks.append(Check.no_duplicate_rows().set_check_name("NoDuplicateFlights"))

# --- 'date' column checks ---
# Ensure 'date' is not null
checks.append(Check.not_null("date").set_check_name("DateNotNull"))
# Ensure 'date' falls within a reasonable historical range (e.g., 2000-2025)
checks.append(Check.column_values_between("date", "2000-01-01", "2025-12-31",
                                          is_expression=True).set_check_name("DateWithinRange"))

# --- 'delay' column checks ---
# Ensure 'delay' is not null
checks.append(Check.not_null("delay").set_check_name("DelayNotNull"))
# Ensure 'delay' is non-negative (delays can't be negative in this context)
checks.append(Check.column_values_greater_than_or_equal_to("delay", 0).set_check_name("DelayNonNegative"))
# Ensure 'delay' is within a realistic range (e.g., max 1440 minutes = 24 hours)
checks.append(Check.column_values_between("delay", 0, 1440).set_check_name("DelayRealisticRange"))


# --- 'distance' column checks ---
# Ensure 'distance' is not null
checks.append(Check.not_null("distance").set_check_name("DistanceNotNull"))
# Ensure 'distance' is positive (a flight must cover a distance)
checks.append(Check.column_values_greater_than("distance", 0).set_check_name("DistancePositive"))


# --- 'origin' and 'destination' column checks ---
# Ensure 'origin' is not null
checks.append(Check.not_null("origin").set_check_name("OriginNotNull"))
# Ensure 'destination' is not null
checks.append(Check.not_null("destination").set_check_name("DestinationNotNull"))

# Example: Check if origin/destination values are from a known set of major airports (illustrative)
# For a real scenario, you'd load this from a lookup table.
major_airports = ["SFO", "LAX", "ORD", "JFK", "ATL", "DFW", "DEN", "CLT", "SEA", "LAS"]
checks.append(Check.column_values_in("origin", major_airports).set_check_name("OriginInMajorAirports"))
checks.append(Check.column_values_in("destination", major_airports).set_check_name("DestinationInMajorAirports"))


print(f"Defined {len(checks)} data quality checks.")

# COMMAND ----------
# MAGIC %md
# MAGIC ## 5. Run Data Quality Checks
# MAGIC
# MAGIC Now, we'll execute the defined checks against our `departure_delays` table.

# COMMAND ----------

# Run the checks on the temporary view 'departure_delays'
results = dq_api.run_quality_checks(
    table_name="departure_delays",
    checks=checks
)

print("Data Quality Checks Executed.")

# COMMAND ----------
# MAGIC %md
# MAGIC ## 6. Display Results Summary
# MAGIC
# MAGIC You can inspect the `results` object to see the outcome of each check.

# COMMAND ----------

# Display summary results
results_df = results.get_results_df()
results_df.display()

# COMMAND ----------
# MAGIC %md
# MAGIC ## 7. Generate and Display DQX Dashboard
# MAGIC
# MAGIC One of the most powerful features of DQX is its ability to generate an interactive HTML dashboard. This dashboard provides a visual summary of your data quality results, making it easy to identify issues.
# MAGIC
# MAGIC The dashboard will be saved to a DBFS path, which you can then download to view in your browser.

# COMMAND ----------

import os

# Define a path where the dashboard HTML file will be saved.
# Using a temporary directory within DBFS for simplicity.
dashboard_output_path = "/dbfs/tmp/dqx_dashboard/flight_delays_dashboard.html"

# Ensure the directory exists
os.makedirs(os.path.dirname(dashboard_output_path), exist_ok=True)

# Generate the dashboard
dashboard_html_path = dq_api.generate_dashboard(
    quality_check_results=results,
    output_path=dashboard_output_path,
    output_format="html"
)

print(f"DQX Dashboard generated at: {dashboard_html_path}")
print(f"To view the dashboard, you can download it from this DBFS path using Databricks UI (Data -> DBFS browser) or Databricks CLI:")
print(f"  dbfs:/tmp/dqx_dashboard/flight_delays_dashboard.html")

# You might be able to display it directly in the notebook if the environment supports it,
# but downloading and opening in a browser provides the best interactive experience.
# Example of displaying partial content for demonstration (might not be fully interactive in all notebooks)
# with open(dashboard_output_path, 'r') as f:
#   html_content = f.read()
# displayHTML(html_content)

# COMMAND ----------